# **4.2 - Image Classification of Dogs vs. Cats vs. Horses**
**15 points**



### **Goals**

4.2 - Know how to use weights and biases with neural networks

4.4 - Understand backwards propogation

4.5 - Understand how to prevent overfitting

4.6 - Know how to interpret loss data/graphs

### **Step 1 -** **Import libraries**
Completed for you

In [ ]:
# put your step 1 code here - done for you
import os
import zipfile
import random
import tensorflow as tf
print(tf.test.gpu_device_name())

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

/device:GPU:0


### **Step 2 - Download two sets of images (dogs and cats, horses)**

**Extension idea: Add humans or other animals or use different ones (Tigers vs Lions vs Cheetahs?)

Completed for you

In [ ]:
# put your step 2 code here - completed for you

# Cats and Dogs
!wget -nc --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "cats-and-dogs.zip"

# Horses
!gdown --id  11NB8Yu0D158WSHms_JHYzsA_g7EOWjRm

--2021-11-12 07:11:17--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.84.168.111, 2600:1407:b800:18a::e59, 2600:1407:b800:19c::e59
Connecting to download.microsoft.com (download.microsoft.com)|184.84.168.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘cats-and-dogs.zip’

cats-and-dogs.zip   100%[===================>] 786.68M   143MB/s    in 5.5s    

2021-11-12 07:11:23 (143 MB/s) - ‘cats-and-dogs.zip’ saved [824894548/824894548]

Downloading...
From: https://drive.google.com/uc?id=11NB8Yu0D158WSHms_JHYzsA_g7EOWjRm
To: /content/horsedataset.zip
100% 77.5M/77.5M [00:00<00:00, 166MB/s]


### **Step 3 - Extract files and move into sub folders**
2 points

In [ ]:
# put your step 3 code here - completed for you

# Cats and Dogs
local_zip = 'cats-and-dogs.zip'
zip_ref  = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('')
zip_ref.close()

# Horses
local_zip = 'horsedataset.zip'
zip_ref  = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('horse')
zip_ref.close()

In [ ]:
!mv /content/PetImages/ /content/Images/
!mv /content/horse/ /content/Images/


# **Step 4 - Balance categories**

**Optional extension**

If the categories are out of balance, the model won't work correctly.

We can trim them to balance the classes by ensuring the same number of images in each folder.

In [ ]:
# put your step 4 code here
# balance up the directories
!rm /content/Images/Dog/[1-9]???.jpg 
!rm /content/Images/Dog/1????.jpg | wc -l
!rm /content/Images/Dog/[0-5]??.jpg
!rm /content/Images/Cat/[1-9]???.jpg 
!rm /content/Images/Cat/1????.jpg | wc -l
!rm /content/Images/horse/07????.png| wc -l
!rm /content/Images/horse/06_09?.png| wc -l
# try running model with and without balancing
# count the number of files in each directory
!ls -l /content/Images/Cat/ | wc -l
!ls -l /content/Images/Dog/ | wc -l
!ls -l /content/Images/horse/ | wc -l






0
0
0
0
1002
502
500


## **Step 5 - Check for corrupt files and remove if necessary**
3 points


In [ ]:
# put your step 5 code here
import os
import cv2
bad_list=[]
dir=r'/content/Images/'
class_list=os.listdir(dir) # create a list of the sub directories in the directory ie train or test
print (class_list)
for klass in class_list: # iterate through the two classes
            class_path=os.path.join(dir, klass) # path to class directory
            #print(class_path)
            file_list=os.listdir(class_path) # create list of files in class directory
            for f in file_list: # iterate through the files
                fpath=os.path.join (class_path,f)
                index=f.rfind('.') # find index of period infilename
                ext=f[index+1:] # get the files extension
                if ext  not in ['jpg', 'png', 'bmp', 'gif']:
                    print(f'file {fpath}  has an invalid extension {ext}')
                    bad_list.append(fpath)                    
                else:
                    try:
                        img=cv2.imread(fpath)
                        size=img.shape
                    except:
                        print(f'file {fpath} is not a valid image file ')
                        bad_list.append(fpath)
                       
print (bad_list)
for image in bad_list:
  os.remove(image)               









['horse', 'Cat', 'Dog']
file /content/Images/Cat/850.jpg is not a valid image file 
file /content/Images/Cat/936.jpg is not a valid image file 
file /content/Images/Cat/140.jpg is not a valid image file 
file /content/Images/Cat/666.jpg is not a valid image file 
file /content/Images/Cat/660.jpg is not a valid image file 
file /content/Images/Cat/Thumbs.db  has an invalid extension db
file /content/Images/Dog/Thumbs.db  has an invalid extension db
['/content/Images/Cat/850.jpg', '/content/Images/Cat/936.jpg', '/content/Images/Cat/140.jpg', '/content/Images/Cat/666.jpg', '/content/Images/Cat/660.jpg', '/content/Images/Cat/Thumbs.db', '/content/Images/Dog/Thumbs.db']


## **Step 6 - Create the training and validation set from the image directories**


We now have three categories. **What do we need to change here?**

3 points

In [ ]:
# put your step 6 code here
img_height=224
img_width=224
batch_size=250
train_data_dir = "/content/Images/"
train_datagen = ImageDataGenerator(rescale=1./255.,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data


Found 1596 images belonging to 3 classes.
Found 398 images belonging to 3 classes.


## **Step 7 - Load the VGG16 model and print model summary**
2 points


In [ ]:
# put your step 7 code here
from tensorflow.keras.applications.vgg16 import VGG16
# load model
model = VGG16() 
# summarize the model
model.summary()




553476096/553467096 [==============================] - 7s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

## **Step 8 - Define and compile our version of the model and print model summary**

**What do we need to change here compared to Cats vs Dogs?**

2 points

In [ ]:
# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(256, activation='relu', kernel_initializer='he_uniform')(flat1)
 
  # output
	output = Dense(3, activation='softmax')(class1)
 
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
 
  # compile model
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	model.summary()
	return model

## **Step 9 - Plot learning curves so we can analyze loss and accuracy**

This is useful as we can see how the model performs as the number of epochs increase.

1 point

In [ ]:
# put your step 9 code here
# plot diagnostic learning curves
def summarize_diagnostics(history):
  # plot loss
  pyplot.subplot(211)
  pyplot.title('Cross Entropy Loss')
  pyplot.plot(history.history['loss'], color='blue', label='train')
  #pyplot.plot(history.history['val_loss'], color='orange', label='test')
  # plot accuracy
  pyplot.subplot(212)
  pyplot.title('Classification Accuracy')
  pyplot.plot(history.history['accuracy'], color='blue', label='train')
 # pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
  # save plot to file
  filename = sys.argv[0].split('/')[-1]
  pyplot.savefig(filename + '_plot.png')
  pyplot.show()
  pyplot.close()




## **Step 10 - Train our version of the model**

Completed for you

In [ ]:
# put your step 10 code here
# done for you
import sys
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = define_model()

nb_epochs=5
history=model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)


58900480/58889256 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/5


## **Step 11 - Show the accuracy of our version of the model**

1 point

In [ ]:
# put your step 11 code here

_, acc = model.evaluate(validation_generator, steps=len(validation_generator), verbose=0)
print('> %.3f' % (acc * 100.0))



## **Step 12 - Summarize the performance of our model**

1 point

In [ ]:
# put your step 12 code here
summarize_diagnostics(history)




# **Step 13 - Check if your model categorizes correctly**

Completed for you

In [ ]:
# Here's a codeblock just for fun. You should be able to upload an image here 
# and have it classified without crashing
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(224, 224))
  pyplot.imshow(img)
  pyplot.show()

  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes)
  print(classes[0])
  print(validation_generator.class_indices)
  i = 0

  for label in validation_generator.class_indices:
        print("\t%s ==> %f" % (label, classes[0][i]))
        i = i + 1